In [4]:
!pip install vtk numpy tqdm


In [ ]:
import vtk
import numpy as np

def load_vti(filename):
    reader = vtk.vtkXMLImageDataReader()
    reader.SetFileName(filename)
    reader.Update()

    image_data = reader.GetOutput()
    dims = image_data.GetDimensions()  # (nx, ny, nz)

    # Extract scalar arrays
    point_data = image_data.GetPointData()
    mean_array = point_data.GetArray("Pressure_Mean")
    std_array = point_data.GetArray("Pressure_Std")

    n_points = mean_array.GetNumberOfTuples()
    mean_np = np.array([mean_array.GetValue(i) for i in range(n_points)])
    std_np = np.array([std_array.GetValue(i) for i in range(n_points)])

    # Reshape to 3D
    mean_3d = mean_np.reshape(dims, order="F")
    std_3d = std_np.reshape(dims, order="F")

    return mean_3d, std_3d, dims


In [ ]:
from tqdm import tqdm

def estimate_crossing_prob(mu_vals, sigma_vals, isovalue, num_samples=1000):
    samples = np.random.randn(num_samples, 8) * sigma_vals + mu_vals
    above = np.any(samples > isovalue, axis=1)
    below = np.any(samples < isovalue, axis=1)
    crossing = above & below
    return np.mean(crossing)

def probabilistic_marching_cubes(mean_3d, std_3d, isovalue, num_samples=1000):
    nx, ny, nz = mean_3d.shape
    prob_vol = np.zeros((nx - 1, ny - 1, nz - 1))

    for i in tqdm(range(nx - 1), desc="PMC"):
        for j in range(ny - 1):
            for k in range(nz - 1):
                corners = [(i+di, j+dj, k+dk) for di in [0, 1] for dj in [0, 1] for dk in [0, 1]]
                mu_vals = np.array([mean_3d[x, y, z] for x, y, z in corners])
                sigma_vals = np.array([std_3d[x, y, z] for x, y, z in corners])
                prob = estimate_crossing_prob(mu_vals, sigma_vals, isovalue, num_samples)
                prob_vol[i, j, k] = prob

    return prob_vol


In [ ]:
def save_prob_vti(prob_vol, output_filename="probability.vti"):
    nx, ny, nz = prob_vol.shape
    image_data = vtk.vtkImageData()
    image_data.SetDimensions(nx, ny, nz)
    image_data.SetSpacing(1.0, 1.0, 1.0)
    image_data.SetOrigin(0, 0, 0)

    # Convert to VTK array
    flat = prob_vol.flatten(order="F")
    vtk_array = vtk.vtkFloatArray()
    vtk_array.SetName("probability")
    vtk_array.SetNumberOfValues(flat.size)
    for i, val in enumerate(flat):
        vtk_array.SetValue(i, val)

    image_data.GetPointData().AddArray(vtk_array)

    writer = vtk.vtkXMLImageDataWriter()
    writer.SetFileName(output_filename)
    writer.SetInputData(image_data)
    writer.Write()


In [ ]:
mean_3d, std_3d, dims = load_vti("isabel_gaussian.vti")
isovalue = 930.0

prob_vol = probabilistic_marching_cubes(mean_3d, std_3d, isovalue, num_samples=2000)
save_prob_vti(prob_vol, "isabel_probability.vti")


PMC: 100%|██████████| 249/249 [37:28<00:00,  9.03s/it]


In [ ]:
import vtk

def print_vti_arrays(filename):
    reader = vtk.vtkXMLImageDataReader()
    reader.SetFileName(filename)
    reader.Update()

    image_data = reader.GetOutput()
    point_data = image_data.GetPointData()

    print("Available scalar arrays:")
    for i in range(point_data.GetNumberOfArrays()):
        print("-", point_data.GetArrayName(i))

print_vti_arrays("/content/isabel_gaussian.vti")


Available scalar arrays:
- Pressure_Mean
- Pressure_Std


In [5]:
import numpy as np
import vtk
from vtk.util.numpy_support import vtk_to_numpy, numpy_to_vtk
from tqdm import tqdm

# === CONFIG ===
INPUT_VTI = "/content/new_tear_pred_150_c.vti"
MEAN_NAME = "mean"
VAR_NAME = "variance"
ISOLEVEL =  159.9798
N_SAMPLES = 1000
OUTPUT_VTI = "159_150_isourfaces_combined.vti"
CORRELATION = 0 # Synthetic correlation coefficient for adjacent cell vertices

# === Load VTI File ===
reader = vtk.vtkXMLImageDataReader()
reader.SetFileName(INPUT_VTI)
reader.Update()
image = reader.GetOutput()
dims = image.GetDimensions()

mean_arr = vtk_to_numpy(image.GetPointData().GetArray(0))
var_arr= vtk_to_numpy(image.GetPointData().GetArray(1))

mean_3d = mean_arr.reshape(dims[::-1], order='F')
std_3d = var_arr.reshape(dims[::-1], order='F')

# === Cell Vertex Indexing ===
def get_cell_vertices(i, j, k):
    return [
        (i,   j,   k),
        (i+1, j,   k),
        (i+1, j+1, k),
        (i,   j+1, k),
        (i,   j,   k+1),
        (i+1, j,   k+1),
        (i+1, j+1, k+1),
        (i,   j+1, k+1),
    ]

# === Build synthetic 8x8 covariance matrix ===
def build_covariance(sigma, rho=0.6):
    cov = np.full((8, 8), rho)
    np.fill_diagonal(cov, 1.0)
    return (sigma[:, None] * sigma[None, :]) * cov

# === Check if isosurface crosses the cell ===
def is_crossing(values, isovalue):
    return np.min(values) < isovalue and np.max(values) > isovalue

# === Monte Carlo Estimation (with correlation) ===
def sample_crossing_probability(mu, sigma, isovalue, n_samples, rho):
    cov = build_covariance(sigma, rho)
    try:
        samples = np.random.multivariate_normal(mu, cov, size=n_samples)
    except np.linalg.LinAlgError:
        # fallback in case of numerical issues
        samples = np.random.normal(loc=mu, scale=sigma, size=(n_samples, 8))
    cross = np.any(samples < isovalue, axis=1) & np.any(samples > isovalue, axis=1)
    return np.mean(cross)

# === Main Computation ===
output_shape = tuple(d - 1 for d in mean_3d.shape)
prob_grid = np.zeros(output_shape, dtype=np.float32)

total_cells = np.prod(output_shape)
progress = tqdm(total=total_cells, desc="Processing Cells")

print("Computing probabilities with correlation...")
for i in range(output_shape[0]):
    for j in range(output_shape[1]):
        for k in range(output_shape[2]):
            verts = get_cell_vertices(i, j, k)
            mu = np.array([mean_3d[x, y, z] for (x, y, z) in verts], dtype=np.float64)
            sigma = np.array([std_3d[x, y, z] for (x, y, z) in verts], dtype=np.float64)
            prob = sample_crossing_probability(mu, sigma, ISOLEVEL, N_SAMPLES, CORRELATION)
            prob_grid[i, j, k] = prob
            progress.update(1)

progress.close()

# === Convert to VTK ImageData ===
prob_image = vtk.vtkImageData()
prob_image.SetDimensions(prob_grid.shape)
prob_image.SetSpacing(image.GetSpacing())
prob_image.SetOrigin(image.GetOrigin())

flat = prob_grid.ravel(order='F')  # VTK expects Fortran order
vtk_array = numpy_to_vtk(flat, deep=True)
vtk_array.SetName("crossing_probability")
prob_image.GetPointData().AddArray(vtk_array)
prob_image.GetPointData().SetActiveScalars("crossing_probability")

# === Write Output ===
writer = vtk.vtkXMLImageDataWriter()
writer.SetFileName(OUTPUT_VTI)
writer.SetInputData(prob_image)
writer.Write()

print(f"Saved output to {OUTPUT_VTI}")


Processing Cells:   0%|          | 388/250047 [00:00<02:08, 1949.90it/s]

Computing probabilities with correlation...


Processing Cells: 100%|██████████| 250047/250047 [02:30<00:00, 1662.07it/s]

Saved output to 159_150_isourfaces_combined.vti
